In [22]:
!pip install thefuzz
!pip install Texttable

In [1]:
# %%
from thefuzz import fuzz
from thefuzz import process
import pandas as pd 
from texttable import Texttable


In [2]:
# %%
#Read in the wootware csv with pandas 
dream = pd.read_csv('../Data/0_DreamWareTech.csv')
evetech = pd.read_csv('../Data/1_Evetech.csv')
rebel = pd.read_csv('../Data/2_Rebeltech.csv')
takealot = pd.read_csv('../Data/3_Takealot.csv')
woot = pd.read_csv('../Data/4_Wootware.csv')
progenix = pd.read_csv('../Data/5_Progenix.csv')
titanice = pd.read_csv('../Data/6_Titanice.csv')


In [7]:
# %%
"""
Formatting tool
"""
def format_title(title):
    title = title.lower()
    title = title.replace('-', ' ')
    title = title.replace(',', ' ')
    title = title.replace('.', ' ')
    title = title.replace('(', '')
    title = title.replace(')', '')
    title = title.replace('[', '')
    title = title.replace(']', '')
    return title


"""
Return a dataframe with the product that best matches the search term.
None if the product is less than tol
"""
def search_product(search, df, tol):
    search = format_title(search)
    # search_result = process.extractOne(search, df['Title'], scorer=fuzz.token_set_ratio)
    matches = process.extract(search,  df['Title'], scorer=fuzz.token_set_ratio)
    filtered_matches = [match for match in matches if match[1] >= tol]


    matched_data = df[df['Title'].isin([match[0] for match in filtered_matches])]
    sorted_data = matched_data.sort_values(by='Price', ascending=True)
    
    if not sorted_data.empty:
        return sorted_data.iloc[0]
    else:
        return None

def search_all_stores(search_term, tol = 95):
    return  {
        'dream': search_product(search_term, dream, tol),
        'evetech': search_product(search_term, evetech, tol),
        'rebel': search_product(search_term, rebel, tol),
        'takealot': search_product(search_term, takealot, tol),
        'woot': search_product(search_term, woot, tol),
        'progenix': search_product(search_term, progenix, tol),
        'titanice': search_product(search_term, titanice, tol),
    }


In [13]:
# %%
search_term = "rtx 3080"

search_result = search_all_stores(search_term=search_term, tol=95)
# print(search_result)
table = [['Store', 'Item', 'Price']]
t = Texttable()
for shop in search_result:
    if shop:

        try:
            table.append([shop.capitalize(), 
                          search_result[shop]['Title'].capitalize(), 
                          str(int(search_result[shop]['Price'])) ])
        except:
            table.append([shop.capitalize(), "N/A", "N/A"])

t.add_rows(table)
print(t.draw())


+----------+-----------------------------------------------------------+-------+
|  Store   |                           Item                            | Price |
+==========+===========================================================+=======+
| Dream    | Gigabyte geforce rtx 3080 eagle 12g gddr6x graphics card  | 18959 |
+----------+-----------------------------------------------------------+-------+
| Evetech  | Msi geforce rtx 3080 gaming x trio 10gb                   | 13999 |
+----------+-----------------------------------------------------------+-------+
| Rebel    | Gigabyte geforce rtx 3080 turbo 10gb gddr6x gv n3080turbo | 16268 |
|          | 10gd graphics card                                        |       |
+----------+-----------------------------------------------------------+-------+
| Takealot | N/A                                                       | N/A   |
+----------+-----------------------------------------------------------+-------+
| Woot     | Gigabyte geforc

In [14]:
from sentence_transformers import SentenceTransformer, util
import torch
def get_best_semantic_match(input_word, product_titles):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Encode the input word and product titles
    input_embedding = model.encode(input_word, convert_to_tensor=True)
    title_embeddings = model.encode(product_titles, convert_to_tensor=True)
    
    # Compute Euclidean distances
    euclidean_distances = torch.cdist(input_embedding.unsqueeze(0), title_embeddings).squeeze(0)
    
    # Find the best match
    best_match_index = euclidean_distances.argmin()
    best_match = product_titles[best_match_index]
    best_match_distance = euclidean_distances[best_match_index].item()
    
    return best_match, best_match_distance

def search_product_1(search, df, tol):
    
    # Example usage
    
    product_titles = df['Title'].tolist()
    # product_titles

    best_match, best_match_distance = get_best_semantic_match(search, product_titles)
    
    return best_match, best_match_distance

def search_all_stores_1(search_term, tol = 95):
    return  {
        'dream': search_product_1(search_term, dream, tol),
        'evetech': search_product_1(search_term, evetech, tol),
        'rebel': search_product_1(search_term, rebel, tol),
        'takealot': search_product_1(search_term, takealot, tol),
        'woot': search_product_1(search_term, woot, tol),
        'progenix': search_product_1(search_term, progenix, tol),
        'titanice': search_product_1(search_term, titanice, tol),
    }

table = [['Store', 'Item', 'Price']]
search_result_1 = search_all_stores_1(search_term=search_term, tol=95)

In [15]:
# Create a texttable object
t = Texttable()
table = [['Shop', 'Product', 'Similarity Score']]

# Populate the table with data
for shop in search_result_1:
    if shop:
        try:
            product, score = search_result_1[shop]
            table.append([shop.capitalize(), product, f"{score:.4f}"])
        except:
            table.append([shop.capitalize(), "N/A", "N/A"])

# Add rows to the table
t.add_rows(table)

# Print the table
print(t.draw())

+----------+------------------------------------------------+------------------+
|   Shop   |                    Product                     | Similarity Score |
+==========+================================================+==================+
| Dream    | gigabyte geforce rtx 3080 eagle 12g gddr6x     | 0.985            |
|          | graphics card                                  |                  |
+----------+------------------------------------------------+------------------+
| Evetech  | msi geforce rtx 3080 gaming x trio 10gb        | 0.922            |
+----------+------------------------------------------------+------------------+
| Rebel    | gigabyte aorus geforce rtx 3080 xtreme         | 0.975            |
|          | waterforce wb 12gb gddr6x gv n3080aorusx wb    |                  |
|          | 12gd graphics card                             |                  |
+----------+------------------------------------------------+------------------+
| Takealot | computer hardwa